# Combinatorial Polyhedron

## Motivation

Given one of Vrepresentation or Hrepresentation it can be a challenging part to determine the other.
Luckily, there are wonderful backends that take care of it.

The double description also gives access to the vertex-facet incidences.
So it should be easy to determine all the related stuff:

- $f$-vector,
- the vertex graph,
- the facet graph,
- $n$-dimensional faces,
- simplicity/simpliciality,
- flag-$f$-vector,
- the face lattice.

Right?

Until recently, any attribute on this list was obtained by the face lattice.
Well, the face lattice can be huge.

In [1]:
P = polytopes.permutahedron(6)
%time P.face_lattice()

CPU times: user 22.3 s, sys: 39.8 ms, total: 22.3 s
Wall time: 22.3 s


Finite lattice containing 4684 elements with distinguished linear extension (use the .plot() method to plot)

This means, everytime you wanted any such information of a polytope of this complexity you had to wait 15 seconds.

No need to mention, that you couldn't deal with large polytopes. No chance you could verify the $20$-dimensional counterexample to the Hirsch conjecture of B. Matschke, F. Santos and C. Weibel with $40$ facets and $36,425$ vertices by calculating the diameter of its graph (see [arXiv:1202.4701](https://arxiv.org/abs/1202.4701)).

With the `CombinatorialPolyhedron` class there is a better implementation of the face lattice, yet to be exposed:

In [2]:
P = polytopes.permutahedron(6)
%time P.combinatorial_polyhedron().face_lattice()

CPU times: user 593 ms, sys: 4.01 ms, total: 597 ms
Wall time: 597 ms


Finite lattice containing 4684 elements (use the .plot() method to plot)

But that's not the point. Many of the above attributes (and probably others) can be obtained without obtaining the full face lattice.



In [3]:
P = polytopes.permutahedron(6)
%time P.f_vector()

CPU times: user 117 ms, sys: 0 ns, total: 117 ms
Wall time: 117 ms


(1, 720, 1800, 1560, 540, 62, 1)

Even better, we know have access to an iterator. You can iterate through the faces of a polyhedron with almost no overhead memory usage.

## Input

`CombinatorialPolyhedron` takes as input (amongst others) one of the following:

- a polyhedron,
- a lattice polytopes,
- a cone,
- an incidence matrix,
- a list of facets, each facet given as list of vertices/rays/lines.

The following example uses the new library of common cones:

In [4]:
C = CombinatorialPolyhedron(cones.rearrangement(3, 8)); C

A 8-dimensional combinatorial polyhedron with 56 facets

In [5]:
%time C.f_vector()

CPU times: user 1.38 ms, sys: 0 ns, total: 1.38 ms
Wall time: 1.39 ms


(1, 1, 16, 112, 392, 770, 840, 420, 56, 1)

## The $f$-vector

To count the number of faces in each dimension, we can utilize a face iterator.

The implementation in sage is the only memory efficient implementation and it is at least as fast any other implementation. It can be parallized without overhead (on-going development) and there are some further significant improvements.

Here are some benchmarks:

In [6]:
P = polytopes.simplex(24)
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 2.12 s, sys: 11.9 ms, total: 2.13 s
Wall time: 2.12 s


(1, 25, 300, 2300, 12650, 53130, 177100, 480700, 1081575, 2042975, 3268760, 4457400, 5200300, 5200300, 4457400, 3268760, 2042975, 1081575, 480700, 177100, 53130, 12650, 2300, 300, 25, 1)

Yes it is trivial. However, this is the first simplex that `polymake` cannot handle with 30 GB of RAM.
`normaliz` needs about 40 seconds parallized on 8 threads (4 cores, hyperthreading).

In [7]:
P = polytopes.associahedron(['A', 10] , backend='normaliz')
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 24.2 s, sys: 296 ms, total: 24.5 s
Wall time: 24.5 s


(1, 58786, 293930, 629850, 755820, 556920, 259896, 76440, 13650, 1365, 65, 1)

`polymake` needs more than 7 hours for this. `normaliz` only 16 seconds (8 threads on 4 cores).
However, sorting the vertices to most significant first, we can speed up by a factor of $2$.
With the ongoing improvements, `sage` will be able to do it in about a second.

In [8]:
P = polytopes.Birkhoff_polytope(5, backend='normaliz')
_ = P.incidence_matrix()
%time P.f_vector()

CPU times: user 537 ms, sys: 3.96 ms, total: 541 ms
Wall time: 540 ms


(1, 120, 5040, 50250, 233400, 631700, 1113700, 1367040, 1220550, 817150, 419225, 167200, 52120, 12600, 2300, 300, 25, 1)

In [9]:
import PyNormaliz
c = P._normaliz_cone
%time PyNormaliz.NmzResult(c, "FVector")

CPU times: user 50.5 s, sys: 604 ms, total: 51.1 s
Wall time: 16.9 s


[1,
 120,
 5040,
 50250,
 233400,
 631700,
 1113700,
 1367040,
 1220550,
 817150,
 419225,
 167200,
 52120,
 12600,
 2300,
 300,
 25,
 1]

Note that this is run on my laptop with only 2 cores. With 4 cores `normaliz` needs about 8 seconds. `polymake` needs almost 6 minutes.

Neither `normaliz` nor 

Taking the $25$-dimensional $6$-Birkhoff polytope both `normaliz` and `polymake` need more than $30$ GB of RAM. Sage needs currently about 45 minutes and much less memory:

In [1]:
start = get_memory_usage()
P = polytopes.Birkhoff_polytope(6)
_ = P.incidence_matrix()
intermediate = get_memory_usage()
it = P.face_generator()
for _ in range(10000):
    a = next(it)
end = get_memory_usage()
print("Total memory: ", end - start, "\nMemory of FaceIterator: ", end - intermediate)

Total memory:  48.3828125 
Memory of FaceIterator:  15.80078125


## Vertex graph/facet graph

In [5]:
P = polytopes.hypercube(10)
C = P.combinatorial_polyhedron()
%time C.vertex_graph(names=False)

CPU times: user 106 ms, sys: 0 ns, total: 106 ms
Wall time: 106 ms


Graph on 1024 vertices (use the .plot() method to plot)

In [6]:
%time C.facet_graph(names=False)

CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 2.17 ms


Graph on 21 vertices (use the .plot() method to plot)

The vertex graph is exposed in `Polyhedron_base`, however only with the graph having the actual vertices as labels. This is much slower:

In [7]:
%time P.vertex_graph()

CPU times: user 3.56 s, sys: 3.91 ms, total: 3.57 s
Wall time: 3.56 s


Graph on 1024 vertices (use the .plot() method to plot)

## $n$-dimensional faces

The main feature of combinatorial polyhedron is a face iterator:

In [11]:
P = polytopes.hypercube(14, backend='field')
C = P.combinatorial_polyhedron()
it = C.face_iter(dimension=5)
%time next(it)

CPU times: user 121 µs, sys: 0 ns, total: 121 µs
Wall time: 124 µs


A 5-dimensional face of a 14-dimensional combinatorial polyhedron

Not specifying the dimension, the iterator will walk through all proper faces:

In [12]:
C = polytopes.simplex().combinatorial_polyhedron()
for i in C.face_iter(): print(i)

A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 2-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron
A 0-dimensional face of a 3-dimensional combinatorial polyhedron
A 1-dimensional face of a 3-dimensional combinatorial polyhedron


In [13]:
C = polytopes.cross_polytope(3).combinatorial_polyhedron()
it = C.face_iter()
[next(it) for _ in range(20)]

[A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 0-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhed

Depending on the number of vertices/facets the iterator will automatically pick wether to start from the vertices or facets (for performance reasons).

However, you can force the behaviour:

In [14]:
it = C.face_iter(dual=False)
[next(it) for _ in range(10)]

[A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 2-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron,
 A 1-dimensional face of a 3-dimensional combinatorial polyhedron]

Each combinatorial face comes with methods similar to polyhedral faces.

In [15]:
f = next(it)
[f.ambient_V_indices(), f.n_ambient_Hrepresentation(), f.ambient_Hrepresentation()]

[(1, 2),
 2,
 (An inequality (1, 1, 1) x + 1 >= 0, An inequality (-1, 1, 1) x + 1 >= 0)]

The face iterator can be manipulated to ignore all subfaces of the current faces in non-dual mode (in dual mode ignore all supfaces). So whenever you see a face and you already know that subfaces are of no interest, then you can spare the time of iterating over them.

A demonstration of with more details is given on [ask.sagemath.org](https://ask.sagemath.org/question/34485/what-is-the-most-efficient-way-to-look-up-a-face-in-the-face-lattice-of-a-polyhedron/?answer=50965#post-id-50965).

`Polyhedron_base` now uses the face iterator for the following methods: `faces`, `face_generator`.

For details of the algorithm see [arXiv:1905.01945](https://arxiv.org/abs/1905.01945)
(joint work with Christian Stump).

## Simplicity/simpliciality

A polyhedron is $k$-simplicial if every $k$-face is a simplex.
A polyhedron is $k$-simple, if its dual is $k$-simplicial.

This can be tested quickly, using the face iterator:

In [16]:
P = polytopes.hypersimplex(12,5)
%time P.simpliciality()

CPU times: user 126 ms, sys: 17 µs, total: 126 ms
Wall time: 126 ms


2

In [17]:
P = polytopes.hypersimplex(12,5)
%time P.simplicity()

CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 130 ms


9

## Face lattice and flag-$f$-vector

Finally, if you really want or need to, you can obtain the face lattice.




This is used to determine e.g. the flag-$f$-vector.

In [18]:
C = polytopes.permutahedron(7).combinatorial_polyhedron()
%time C.face_lattice()

CPU times: user 6.44 s, sys: 27.9 ms, total: 6.47 s
Wall time: 6.46 s


Finite lattice containing 47294 elements (use the .plot() method to plot)

Currently, this is used only to obtain the flag-$f$-vector:

In [19]:
polytopes.hypercube(4).flag_f_vector(0,3)

64